## Bibliotecas

In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, matthews_corrcoef
import random
import numpy as np

## Preparar entorno

In [4]:
# Configurar
seed = 26
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Cargar modelo

In [2]:
# Rutas para el model entrenado y el tokenizador
model_path = "../modelo/"
tokenizer_path = "../modelo/"

In [3]:
# Cargar modelo entrenado y tokenizador
model = RobertaForSequenceClassification.from_pretrained(model_path, num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)

In [5]:
# Configuracion de batch
batch_size = 16

In [6]:
# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Prediccion

In [12]:
# Ejemplo de datos para predecir
# nuevo_titulo = "Moreno intenta apaciguar el flanco sanitario mientras enreda con la fecha de las elecciones"
# nueva_descripcion = "El presidente abre la puerta a unos comicios en junio que no sean en domingo."
# nueva_fecha = "19/04/2022"
nuevo_titulo = "Las promesas incumplidas de Pablo Echenique en sanidad, educación y vivienda"
nueva_descripcion = "Este lunes y martes la Asamblea de Madrid acogerá el debate del estado de la región. El último se celebró hace dos años, el 14 de septiembre de 2020."
nueva_fecha = "12/09/2022"

# Tokeniza el nuevo ejemplo
nuevos_encodings = tokenizer(
    nuevo_titulo,
    nueva_descripcion,
    nueva_fecha,
    padding="max_length",
    truncation='only_second',
    max_length=128,
    return_tensors="pt"
)

# Obtiene los input_ids y attention_masks del nuevo ejemplo
nuevos_input_ids = nuevos_encodings["input_ids"]
nuevas_attention_masks = nuevos_encodings["attention_mask"]

# Mueve los datos a la GPU si está disponible
nuevos_input_ids = nuevos_input_ids.to(device)
nuevas_attention_masks = nuevas_attention_masks.to(device)

# Realiza la predicción
with torch.no_grad():
    logits = model(nuevos_input_ids, attention_mask=nuevas_attention_masks).logits
    _, predicted_label = torch.max(logits, 1)

# Convierte el resultado a una etiqueta legible
etiqueta_predicha = "Noticia falsa" if predicted_label.item() == 0 else "Noticia verdadera"

# Imprime la etiqueta predicha
print(f"[+]Predicción: {etiqueta_predicha}")


[+]Predicción: Noticia falsa
